# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine    

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# Importing the dataset from Ironhack's database
driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'orders'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [4]:
orders = pd.read_sql('SELECT * FROM orders ;', engine)
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

### Identifying total amount spent by CustomerID and Country

In [5]:
#First sum amoutn spent and groupby custoemr id
orders_agg = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':'sum'}).sort_values('CustomerID')
orders_agg

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


### Identifying the aggregate amount spent that falls in both specified quantiles 

In [6]:
orders_agg.quantile([.75, 0.95], axis = 0)

,amount_spent
0.75,1661.195
0.95,5774.278


#### Preferred Customers

They fall in:
- Between the 75th & 95th percentile
- Their aggregate spent amount is equal to or more than 1661

#### VIP Customers

They fall in:
- Above the 95th percentile
- Their aggregate spent amount is equal to or more than 5774

### Assigning the necessary quantiles

In [7]:
q75 = orders_agg.quantile(.75)
q95 = orders_agg.quantile(.95)

### Creating the categories based on the quantiles

In [8]:
condlist = [orders_agg["amount_spent"]>q95[0], orders_agg["amount_spent"]>q75[0]]
choicelist = ["VIP", "Preferred"]
orders_agg["customer_tags"] = np.select(condlist, choicelist, default="Normal")
orders_agg.head()

,,amount_spent,customer_tags
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

### Identify VIP countries

In [9]:
VIP_countries = orders_agg['customer_tags'] == 'VIP'
VIP_customers = orders_agg.loc[VIP_countries]
VIP_customers.head()

,,amount_spent,customer_tags
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12357,Switzerland,6207.67,VIP
12359,Cyprus,6372.58,VIP
12409,Switzerland,11072.67,VIP
12415,Australia,124914.53,VIP


### Group VIPs by country

In [10]:
VIP_customers = VIP_customers.groupby('Country').agg({'customer_tags': 'count'})
VIP_customers.rename(columns={'customer_tags': 'Total VIPs'}, inplace=True)
VIP_customers

,Total VIPs
Country,
Australia,1
Channel Islands,1
Cyprus,1
Denmark,1
EIRE,2
Finland,1
France,9
Germany,11
Japan,2


### Sort by highest num of VIPs

In [11]:
VIP_customers.sort_values('Total VIPs', ascending=False).head(1)

,Total VIPs
Country,
United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
# your code here
countries = (orders_agg['customer_tags'] == 'VIP') | (orders_agg['customer_tags'] == 'Preferred')
idx = orders_agg.loc[countries]
idx.head()

,,amount_spent,customer_tags
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12352,Norway,2506.04,Preferred


In [13]:
important_countires = idx.groupby(['Country']).agg({'amount_spent':'count'})
important_countires.rename(columns={'amount_spent': 'Total VIPs and Preferred'}, inplace=True)
important_countires.sort_values('Total VIPs and Preferred', ascending=False).head(1)

,Total VIPs and Preferred
Country,
United Kingdom,934
